In [1]:
# %load Q4.py
"""
Created on Tue Apr 11 22:34:53 2023

@author: Light Coilgun Batteries
"""

import pandas as pd
import numpy as np
import os

# 
#os.chdir(r'E:\OneDrive\NUS\BT4016 Risk Analytics\Project/')

# BTC & ETH

# a)
### ///-------  volatility of the underlying coin in pct return  -------\\\  ###
BTC_raw = pd.read_csv('BTC_table.csv')  # this is also allocation table from unconstrained
ETH_raw = pd.read_csv('ETH_table.csv')  # this is also allocation table from unconstrained
# ['Date', 'Security', 'notional', 'price_USD', 'market_value_USD']


BTC_raw['price_USD_lag1'] = BTC_raw['price_USD'].shift(1)
BTC_raw['log_return'] = np.log(BTC_raw['price_USD']/BTC_raw['price_USD_lag1'])
BTC_return_vol = BTC_raw['log_return'].std()*365**0.5*100

ETH_raw['price_USD_lag1'] = ETH_raw['price_USD'].shift(1)
ETH_raw['log_return'] = np.log(ETH_raw['price_USD']/ETH_raw['price_USD_lag1'])
ETH_return_vol = ETH_raw['log_return'].std()*365**0.5*100

# total return volatility, non-directional
print('BTC log return volatility annualized pct:',BTC_return_vol)
print('ETH log return volatility annualized pct:',ETH_return_vol)
#  (You can choose to also consider the upside and downside risks separately.)
"""
ETH is more volatile return-wise
"""

# ETH_price_vol = ETH_raw[ETH_raw['Date'] < "2021-01-01"]['price_USD'].std()  # to use in part b
# BTC_price_vol = BTC_raw[BTC_raw['Date'] < "2021-01-01"]['price_USD'].std()  



one_stdev = np.exp(ETH_raw[ETH_raw['Date'] < "2021-01-01"]['log_return'].std())-1
ETH_price_on_1_jan_2021 = ETH_raw[ETH_raw['Date'] == '2021-01-01']['price_USD']
ETH_price_vol = ETH_price_on_1_jan_2021 * one_stdev
ETH_price_vol = ETH_price_vol.values[0]   # tianbo's edit
ETH_price_vol 

BTC log return volatility annualized pct: 78.55740229307276
ETH log return volatility annualized pct: 104.31100675127634


40.354491585030026

In [2]:
one_stdev_BTC = np.exp(BTC_raw[BTC_raw['Date'] < "2021-01-01"]['log_return'].std())-1
BTC_price_on_1_jan_2021 = BTC_raw[BTC_raw['Date'] == '2021-01-01']['price_USD']
BTC_price_vol = BTC_price_on_1_jan_2021 * one_stdev
BTC_price_vol = BTC_price_vol.values[0]   # tianbo's edit
BTC_price_vol

1622.9896543876246

In [3]:
# b)
### ///-------  Straddle Analysis  -------\\\  ###
    # use unconstrained port 
    # load option data and clean, timewise also need to sort
    # buy one straddle (buy one put + buy one call). That expires on 2021/01/08
    #   on the most volatile ticker
    # the straddle can have X = [-1 SD,same] put and [same, +1 SD] call 
"""
    You can choose the exercise prices as close as to minus/plus one 
    standard deviation of asset return (minus one standard deviation for buying a put and plus one 
    standard deviation for buying a call option). You can determine your cost using the last 
    transacted option prices on 2021/01/01 as the market price. Suppose your cost is externally 
    financed and to be returned/deducted on 2021/01/08, with no interest rate. 
    What are the instrument_ids that you decided to buy?
"""

optionDF_raw = pd.read_csv('OKEx_options_trade.csv',index_col=0)
# ['side', 'trade_id', 'price', 'qty', 'instrument_id', 'timestamp']

"""
BTC-USD-210226-7000-P 

Take the first row instrument here for example, in the instrument_id: “BTC-USD” refers to 
the underlying asset, “210226” refers to its expiration date which is 2021/02/26, “US$7000” is 
the strike price.

OKEX options are EUROPEAN options

timestamp: 2020-12-17T12:35:08.298Z
"""

# parsing, get ['ticker','expDate','exPX','type','date'] 
# expDate, expiration date, 
# date, timestamp parsed to date,
# type, "P" or "C"
optionDF_raw['ticker'] = optionDF_raw['instrument_id'].str.split('-').apply(lambda x: x[0])
optionDF_raw['ticker'] = optionDF_raw['ticker']+'-USD'
# unique: ['instrument_id-USD', 'BTC-USD', 'ETH-USD'], clean:
optionDF_raw = optionDF_raw[optionDF_raw.ticker != 'instrument_id-USD'].reset_index(drop=True)

optionDF_raw['expDate'] = optionDF_raw['instrument_id'].str.split('-').apply(lambda x: x[2])
optionDF_raw['expDate'] = pd.to_datetime(optionDF_raw['expDate'], format='%y%m%d')
optionDF_raw['expDate'] = optionDF_raw['expDate'].dt.strftime('%Y-%m-%d')

optionDF_raw['exPX'] = optionDF_raw['instrument_id'].str.split('-').apply(lambda x: x[3]).astype(np.float)
optionDF_raw['type'] = optionDF_raw['instrument_id'].str.split('-').apply(lambda x: x[4])
optionDF_raw['Date'] = optionDF_raw['timestamp'].str.split('T').apply(lambda x: x[0])

# note original price is in corresponding crypto
optionDF_raw['price'] = optionDF_raw['price'].astype(np.float)

optionDF_raw = optionDF_raw.sort_values(by='Date')
print('trade date range',optionDF_raw['Date'].min(),optionDF_raw['Date'].max())
print('exp date range',optionDF_raw['expDate'].min(),optionDF_raw['expDate'].max())
# trade date range 2020-01-27 2021-12-01
# exp date range 2020-02-07 2022-03-25


# find max PnL:
# on 2021/01/01
# buy one straddle (buy one put + buy one call) that expires on 2021/01/08
# You can determine your cost using the last transacted option prices on 2021/01/01 as the market price. !!!
# You can choose the exercise prices as close as to minus/plus one standard deviation of asset return
#   minus one standard deviation for buying a put and plus one for buying a call option
#   running non-retradeable assumption
#   running simple closing price assumption on each contract each day
#   running exPX_P can unequal vs exPX_C assumption
#   running risk-free rate = 0 assumption

# first see ETH price on 2021/01/01 vs 2021/01/08
print(ETH_raw[ETH_raw.Date =='2021-01-01'].price_USD)
print(ETH_raw[ETH_raw.Date =='2021-01-08'].price_USD)
#730.367554 to 1224.197144
# PnL = max(-exPX_C + price_underlying, 0) - PX_C  \
#     + max(+exPX_P - price_underlying, 0) - PX_P
# calculate each best solution separately
# and combine two 
# "You can choose the exercise prices as close as to minus/plus one standard deviation of asset RETURN"
startDate = '2021-01-01' #<----PARAM
endDate = '2021-01-08' #<----PARAM

# reverse merge since option dates might be missing (not trading everyday)
optionDF = optionDF_raw[optionDF_raw.ticker=='ETH-USD']
optionDF = optionDF[optionDF.expDate == endDate]

#filter for options with a price on startDate (tradeable), can keep only startDate data for option
optionDF = optionDF[optionDF.Date==startDate]
# issue: each option could get traded multiple times, this raw file is tick data
optionDF = optionDF.drop_duplicates(subset=['Date','instrument_id'],keep='last')

optionDF.instrument_id.nunique()


# cross-merge on instrument_id vs ETH, so every instrument_id is populated for each Date with all necessary data
# create backbone DF first
df1 = ETH_raw[(ETH_raw.Date >= startDate) & (ETH_raw.Date <= endDate)][['Date']]
df2 = optionDF[['instrument_id']]
optionDF_ts_full = df1[['Date']].merge(df2[['instrument_id']],how='cross')

# need: 
#   price_first: price on startDate, since optionDF only keep startDate price, this is eq. price
#   exPX on startDate
#   ETH price_USD everyday
#   type of option
optionDF_ts_full = optionDF_ts_full.merge(optionDF[['instrument_id','exPX','type','price','expDate']],on='instrument_id'
                                          ,how='left')
optionDF_ts_full = optionDF_ts_full.merge(ETH_raw[['Date','price_USD']],on='Date',how='left')
# note original price is in corresponding crypto, converting
optionDF_ts_full['price_option_USD'] = optionDF_ts_full['price']*optionDF_ts_full['price_USD']

# price of option and price of underlying at purchase
optionDF_ts_full = optionDF_ts_full.sort_values(by='Date')
optionDF_ts_full['price_option_first'] = optionDF_ts_full.groupby('instrument_id')['price_option_USD'].transform('first')
optionDF_ts_full['price_underlying_first'] = optionDF_ts_full.groupby('instrument_id')['price_USD'].transform('first')

print(ETH_price_vol)
# for filtering later # ETH_return_vol # ETH_price_vol
optionDF_ts_full['PX_minus_vol'] = optionDF_ts_full['price_underlying_first'] - ETH_price_vol
optionDF_ts_full['PX_plus_vol'] = optionDF_ts_full['price_underlying_first'] + ETH_price_vol
optionDF_ts_full['zero_dummy'] = 0.0


#   exPX in [price_USD-ETH_vol, price_USD] if P
#   exPX in [price_USD, price_USD+ETH_vol] if C

# filter and keep, PUT
optionDF_P = optionDF_ts_full[optionDF_ts_full['type']=='P']
optionDF_P = optionDF_P[(optionDF_P.Date >= startDate) & (optionDF_P.Date <= endDate)]

optionDF_P["PnL"] = (optionDF_P['exPX'] - optionDF_P['price_USD'])
optionDF_P["PnL"] = optionDF_P[["PnL",'zero_dummy']].max(axis=1) - optionDF_P['price_option_first']

mask_P_lower_bound = optionDF_P['exPX'] >= optionDF_P['PX_minus_vol']
mask_P_upper_bound = optionDF_P['exPX'] <= optionDF_P['price_underlying_first']


print("LOWER BOUIND " , mask_P_lower_bound)
print("UPPER BOUIND " , mask_P_upper_bound)

optionDF_P = optionDF_P[mask_P_lower_bound & mask_P_upper_bound]
optionDF_P_end = optionDF_P[optionDF_P.Date==endDate]

print(optionDF_P_end.reset_index(drop=True).loc[optionDF_P_end["PnL"].argmax(),])
optimal_P_row = optionDF_P_end.reset_index(drop=True).loc[optionDF_P_end["PnL"].argmax(),]


# filter and keep, CALL
optionDF_C = optionDF_ts_full[optionDF_ts_full['type']=='C']
optionDF_C = optionDF_C[(optionDF_C.Date >= startDate) & (optionDF_C.Date <= endDate)]

optionDF_C["PnL"] = (-optionDF_C['exPX'] + optionDF_C['price_USD'])
optionDF_C["PnL"] = optionDF_C[["PnL",'zero_dummy']].max(axis=1) - optionDF_C['price_option_first']

mask_P_lower_bound = optionDF_C['exPX'] >= optionDF_C['price_underlying_first']
mask_P_upper_bound = optionDF_C['exPX'] <= optionDF_C['PX_plus_vol']
optionDF_C = optionDF_C[mask_P_lower_bound & mask_P_upper_bound]
optionDF_C_end = optionDF_C[optionDF_C.Date==endDate]

print(optionDF_C_end.reset_index(drop=True).loc[optionDF_C_end["PnL"].argmax(),])
optimal_C_row = optionDF_C_end.reset_index(drop=True).loc[optionDF_C_end["PnL"].argmax(),]

print('optimal straddle contracts',optimal_P_row.instrument_id,optimal_C_row.instrument_id)


<ipython-input-3-5b18190fd023>:45: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  optionDF_raw['exPX'] = optionDF_raw['instrument_id'].str.split('-').apply(lambda x: x[3]).astype(np.float)
<ipython-input-3-5b18190fd023>:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  optionDF_raw['price'] = optionDF_raw['price'].astype(np.float)


trade date range 2020-01-27 2021-12-01
exp date range 2020-02-07 2022-03-25
334    730.367554
Name: price_USD, dtype: float64
341    1224.197144
Name: price_USD, dtype: float64
40.354491585030026
LOWER BOUIND  18     False
14     False
13      True
12     False
11      True
       ...  
154    False
153     True
151    False
149    False
156    False
Length: 88, dtype: bool
UPPER BOUIND  18      True
14      True
13      True
12      True
11     False
       ...  
154     True
153    False
151     True
149     True
156     True
Length: 88, dtype: bool
Date                                2021-01-08
instrument_id             ETH-USD-210108-700-P
exPX                                     700.0
type                                         P
price                                   0.0255
expDate                             2021-01-08
price_USD                          1224.197144
price_option_USD                     31.217027
price_option_first                   18.624373
price_underlying_fi

In [4]:
# c)
### ///-------  Straddle Simulation  -------\\\  ###
# annualized return and Sharpe ratio
# with no interest rate.
# assuming no portfolio value impact before option exercise

portfolio_df_unrestricted = pd.read_csv('portfolio_df_unrestricted_updated.csv',index_col=0)
portfolio_df_unrestricted
ETH_raw

,Date,Security,notional,price_USD,market_value_USD,price_USD_lag1,log_return
0,2020-02-02,ETH,6.543702e-16,188.617538,1.201908e-13,NaN,NaN
1,2020-02-03,ETH,8.301673e-16,189.865067,1.565841e-13,188.617538,0.006592
2,2020-02-04,ETH,6.307485e-16,189.250595,1.197571e-13,189.865067,-0.003242
3,2020-02-05,ETH,6.042622e-16,204.230240,1.143570e-13,189.250595,0.076176
4,2020-02-06,ETH,6.305597e-16,212.339081,1.287794e-13,204.230240,0.038936
...,...,...,...,...,...,...,...
726,2022-01-28,ETH,1.115506e-03,2547.092041,2.702873e+00,2423.001221,0.049945
727,2022-01-29,ETH,1.010494e-03,2597.084717,2.573822e+00,2547.092041,0.019437
728,2022-01-30,ETH,1.399140e-03,2603.466553,3.633684e+00,2597.084717,0.002454
729,2022-01-31,ETH,1.492258e-03,2688.278809,3.885043e+00,2603.466553,0.032057


In [5]:
#2020-2-1 to 2022-2-1
portfolio_df_unrestricted['Date'] = ETH_raw['Date']


# without straddle
portfolio_df_unrestricted_original = portfolio_df_unrestricted.copy()

# with straddle, option size - one straddle for one unit underlying asset
portfolio_df_unrestricted_straddled = portfolio_df_unrestricted.copy()

notional_underlying = ETH_raw[ETH_raw.Date ==startDate].notional.values[0]
print('underlying coin quantity of unrestricted port on',startDate,notional_underlying)
straddle_PnL = (optimal_P_row.PnL + optimal_C_row.PnL) * notional_underlying

portfolio_df_unrestricted_straddled.loc[portfolio_df_unrestricted_straddled.Date == endDate,'Portfolio_value'] += straddle_PnL

portfolio_df_unrestricted_straddled['Prev value'] = portfolio_df_unrestricted_straddled['Portfolio_value'].shift(1)
portfolio_df_unrestricted_straddled['log_return'] = np.log(portfolio_df_unrestricted_straddled['Portfolio_value']
                                                           /portfolio_df_unrestricted_straddled['Prev value'])
portfolio_df_unrestricted_straddled['perc_return'] = (portfolio_df_unrestricted_straddled['Portfolio_value']
                                                      /portfolio_df_unrestricted_straddled['Prev value']) - 1


annualised_returns_unrestricted_original = np.prod(1 + portfolio_df_unrestricted_original['perc_return']) ** (1/730)  * 365 - 1
annualised_volatility_unrestricted_original = portfolio_df_unrestricted_original['log_return'].std()* 365 ** 0.5 * 100
sharpe_ratio_unrestricted_original = annualised_returns_unrestricted_original/annualised_volatility_unrestricted_original
print("annualised % return of original unrestricted portfolio is "  + str(annualised_returns_unrestricted_original))
print("sharpe ratio of original unrestricted portfolio is "  + str(sharpe_ratio_unrestricted_original))

annualised_returns_unrestricted_straddled = np.prod(1 + portfolio_df_unrestricted_straddled['perc_return']) ** (1/730)  * 365 - 1
annualised_volatility_unrestricted_straddled = portfolio_df_unrestricted_straddled['log_return'].std()* 365 ** 0.5 * 100
sharpe_ratio_unrestricted_straddled = annualised_returns_unrestricted_straddled/annualised_volatility_unrestricted_straddled
print("annualised % return of straddled unrestricted portfolio is "  + str(annualised_returns_unrestricted_straddled))
print("sharpe ratio of straddled unrestricted portfolio is "  + str(sharpe_ratio_unrestricted_straddled))


underlying coin quantity of unrestricted port on 2021-01-01 -2.2752596930398483e-16
annualised % return of original unrestricted portfolio is 364.98910401118593
sharpe ratio of original unrestricted portfolio is 4.475949489477862
annualised % return of straddled unrestricted portfolio is 364.98910401118593
sharpe ratio of straddled unrestricted portfolio is 4.4759494894778555
